## Machine Learning Assignment 2

#### Group 1: Siow Meng Low, Louise Fallon, Nikhita Venkatesan, George Pastakas, Cecilia Nok Sze Cheung, Steven Locorotondo

#### Group Assignment: Creating a SMS Message Spam Filter

In this assignment we will use the data file [SMSSpamCollection](https://archive.ics.uci.edu/ml/machine-learning-databases/00228/) which contains SMS messages and categorises them in two types, *ham* or *spam*. Using this data, we will build a Naive Bayes spam filter.

In [1]:
import numpy as np
import pandas as pd
import string
from sklearn.model_selection import train_test_split

seedValue = 99

**1.** First, we load the data into a Python data frame.

The **read_table** function is used to read the tab separated file. 

Note that within the data file, there are double quotation marks which span from Line 5082 to Line 5084. If we use the default **quoting** option in **read_table**, the three lines will be read in as a single record. In order to read these 3 lines as separate records, we set **quoting = 3** to instruct the reader not to perform special processing of quote characters. 

Total number of records read is 5574 messages.

In [2]:
data = pd.read_table('./smsspamcollection/SMSSpamCollection', names = ['Type', 'Message'], quoting = 3)

**2.** Before moving on, we will pre-process the SMS messages, by:

1. Removing all punctuation and numbers from the SMS messages
2. Changing all messages to lower case

The following code replaces all the uppercase characters to lowercase. It also removes all the punctuation characters, digits and pound sterling symbol.

In [3]:
translator = str.maketrans(string.ascii_uppercase, 
                           string.ascii_lowercase, 
                           string.punctuation + string.digits + '£')
data['Clear Message'] = data['Message'].str.translate(translator)

**3.** Next, we shuffle the messages and split them into

* a training set (2,500 messages)
* a validation set (1,000 messages) and 
* a test set (2,074 messages)

The scikit-learn function 'train_test_split' below does the record shuffling before splitting into separate sets.

In [4]:
# Shuffle the records and randomly select 2,500 messages out of the 5,574 as train set, the rest as "other" set
train, others = train_test_split(data, train_size = 2500, random_state = seedValue)
# Randomly select 1,000 messages from the 3,074 "other" messages as validation set, remaining 2,074 as "test" set
validation, test = train_test_split(others, train_size = 1000, random_state = seedValue)


**4.** While Python’s SciKit-Learn library has a Naive Bayes classifier, it works with continuous probability distributions and assumes numerical features. Although it is possible to transform categorical variables into numerical features using a binary encoding, we will instead build a simple Naive Bayes classifier from scratch, which includes the following functions:

* `train()`
* `train2()`
* `predict()`
* `score()`

In [5]:
class NaiveBayesForSpam:
  
    def train(self, hamMessages, spamMessages):
        self.words = set(' '.join(hamMessages + spamMessages).split())
        self.priors = np.zeros(2)
        self.priors[0] = float(len(hamMessages)) / (len(hamMessages) + len(spamMessages))
        self.priors[1] = 1.0 - self.priors[0]
        self.likelihoods = []
        for i, w in enumerate(self.words):
            prob1 = (1.0 + len([m for m in hamMessages if w in m])) / len(hamMessages)
            prob2 = (1.0 + len([m for m in spamMessages if w in m])) / len(spamMessages)
            self.likelihoods.append([min(prob1, 0.95), min(prob2, 0.95)])
        self.likelihoods = np.array(self.likelihoods).T

    def train2(self, hamMessages, spamMessages):
        self.words = set(' '.join(hamMessages + spamMessages).split())
        self.priors = np.zeros(2)
        self.priors[0] = float(len(hamMessages)) / (len(hamMessages) + len(spamMessages))
        self.priors[1] = 1.0 - self.priors[0]
        self.likelihoods = []
        spamkeywords = []
        for i, w in enumerate(self.words):
            prob1 = (1.0 + len([m for m in hamMessages if w in m])) / len(hamMessages)
            prob2 = (1.0 + len([m for m in spamMessages if w in m])) / len(spamMessages)
            if prob1 * 20 < prob2:
                self.likelihoods.append([min(prob1, 0.95), min(prob2, 0.95)])
                spamkeywords.append(w)
        self.words = spamkeywords
        self.likelihoods = np.array(self.likelihoods).T
    
    def predict(self, message):
        posteriors = np.copy(self.priors)
        for i, w in enumerate(self.words):
            if w in message.lower(): # convert to lower-case
                posteriors *= self.likelihoods[:, i]
            else:
                posteriors *= np.ones(2) - self.likelihoods[:, i]
            posteriors = posteriors / np.linalg.norm(posteriors, ord = 1) # normalise
        if posteriors[0] > 0.5:
            return ['ham', posteriors[0]]
        return ['spam', posteriors[1]]
    
    def score(self, messages, labels):
        confusion = np.zeros(4).reshape(2, 2)
        for m, l in zip(messages, labels):
            if self.predict(m)[0] == 'ham' and l == 'ham':
                confusion[0, 0] += 1
            elif self.predict(m)[0] == 'ham' and l == 'spam':
                confusion[0, 1] += 1
            elif self.predict(m)[0] == 'spam' and l == 'ham':
                confusion[1, 0] += 1
            elif self.predict(m)[0] == 'spam' and l == 'spam':
                confusion[1, 1] += 1
        return (confusion[0, 0] + confusion[1, 1]) / float(confusion.sum()), confusion

**5.** Explain the Code

The functions used in `class NaiveBayesForSpam` are:

#### `train()`

The input arguments of `train()` function are two lists of strings: `hamMessages` and `spamMessages` containing the ham messages and spam messages, respectively. 

It first merges these two lists and creates a set of all the unique words contained in any of the messages (regardless of whether the message is ham or spam). Next, it derives the two prior probabilites $P(ham)$ and $P(spam)$ by computing the proportion of ham and spam messages. The prior probabilities are stored in a list of length 2, named `priors`.

After that, for each word $W$ in the set of words, the function calculates the likelihood of each word of being in ham or spam message, i.e. $P(W \mid ham)$ and $P(W \mid spam)$ for each $W$. 1 is added to the number of occurrences of each word in ham and spam emails to avoid having probabilities equal to zero (Laplace estimator). 

Finally, it caps the values of the calculated probabilities at 0.95 by replacing probabilities larger than this value with 0.95. This is to prevent the negations (i.e. $P(\neg W \mid ham)$ and $P(\neg W \mid spam)$) from being zero. The result is stored in the list `likelihoods` of length equal to the number of unique words.

#### `train2()`

The `train2()` function takes the same input arguments as function `train()` and has the same purpose. The only difference is that it only stores the spam keywords. Spam keywords are those words which have a probability of being in spam messages at least 20 times higher than a probability of encountered in ham messages, i.e. $20 \times P(W \mid ham) < P(W \mid spam)$. Again, it caps the likelihood values of these words at 0.95 by replacing probabilities larger than this value with 0.95. The final list of predictor words only consists of these spam keywords.

####  `predict()`

The `predict()` function takes a single string as input argument, `message`, which represents a new SMS message that needs to be classified as ham or spam. First, it copies the values of the two prior probabilities $P(ham)$ and $P(spam)$, into the array named `posteriors`, which will be used to calculate the posterior probabilities of the new message being ham or spam. According to Bayes' Theorem:

$$P(ham \mid message) \propto P(messagen \mid ham) \times P(ham)$$

and
$$P(spam \mid message) \propto P(messagen \mid spam) \times P(spam)$$

where the Naive Bayes classifiers assume class-conditional independence and hence:

$$P(message \mid ham) = P(W_1 \mid ham) \times P(W_2 \mid ham) \times \cdots \times P(W_n \mid ham)$$

and
$$P(message \mid spam) = P(W_1 \mid spam) \times P(W_2 \mid spam) \times \cdots \times P(W_n \mid spam)$$

if the message contains all the words $W_1, W_2, ..., W_n$. For the words $W_i$ that are not contained in the message $P(W_i \mid ham)$ are to be replaced as $P(\neg W_i \mid ham) = 1 - P(W_i \mid ham)$ and $P(W_i \mid spam)$ are to be replaced as $P(\neg W_i \mid spam) = 1 - P(W_i \mid spam)$. 

The function does this Right Hand Side computation of Bayes' Theorem formula. It initialises using the two prior probabilities $P(ham)$ and $P(spam)$. Then it iterates through all the words we have in our classifier and checks whether each word $W_i$ exists in the new message or not. If so, it multiplies the two probabilities with $P(W_i \mid ham)$ and $P(W_i \mid spam)$ respectively. Otherwise it multiplies them with $P(\neg W_i \mid ham) = 1 - P(ham \mid W_i)$ and $P(\neg W_i \mid spam) = 1 - P(spam \mid W_i)$, respectively. After each multiplication, the function performs a normalisation step so that the two probabilities sum to one. This is to prevent having very small values in the posterior probabilities (which may lead to numerical issues for computers). After all the multiplications and final normalisation, the final values calculated are the posterior probabilities, $P(ham \mid message)$ and $P(spam \mid message)$.

If the posterior probability of being a ham, $P(ham \mid message)$ is greater than 0.5, the function classifies the new message as ham. Otherwise, it classifies it as spam. Both the predicted class and the posterior probability (of the predicted class) are returned.

#### `score()`

The input arguments of the `score()` function are two lists of strings, `messages` which contain the messages which we would like to classify and `labels` which contains the true labels of the messages. 

Firstly, it first creates a $2 \times 2$ matrix of zeros, which constitutes the confusion matrix. It makes use the `predict` function to make predictions on the classification of each messages. Finally, it compares the prediction against the true label in order to update the values in the confusion matrix:

* $predicted = ham$ and $actual = ham$: Increase ***True Negatives*** (***TN***) by 1 (top left hand corner of the matrix)
* $predicted = ham$ and $actual = spam$: Increase ***False Negatives*** (***FN***) by 1 (top right hand corner of the matrix)
* $predicted = spam$ and $actual = ham$: Increase ***False Positives*** (***FP***) by 1 (bottom left hand corner of the matrix)
* $predicted = spam$ and $actual = spam$: Increase ***True Positives*** (***TP***) by 1 (bottom right hand corner of the matrix)

Note that for this confusion matrix, the columns indicate the true labels while the rows represent the predicted classes. The locations of **TN, TP, FN, FP** are described in the above bulleted points.

Finally, the function returns both the confusion matrix and the accuracy of the model, which is calculated as

$$accuracy = \frac{TN + TP}{TN + FN + FP + TP}$$

#### **Where is Bayes' Theorem Applied?**

Bayes' Theorem is applied in the predict() function, where the posterior probabilities (of ham or spam) are calculated by multiplying prior probability with the likelihoods of different words in ham or spam message respectively. The final step of normalising the two posterior probabilities so that they sum to one is akin to dividing the two values by marginal likelihood, so that the final calculated values are posterior probabilities (where the sum of two posterior probabilities is one).

**6.** Use your training set to train the classifiers `train()` and `train2()`. Note that the interfaces of our classifiers require you to pass the ham and spam messages separately.

The below code initialises two classifiers, **NB_1** and **NB_2**. **NB_1** uses `train()` function while **NB_2** uses `train2()` for training.

In [6]:
# Separate messages of training set to hams and spams
train_hams = list(train[train["Type"] == "ham"]["Clear Message"])
train_spams = list(train[train["Type"] == "spam"]["Clear Message"])

# Build Naive Bayes classifier using "train()" function
NB_1 = NaiveBayesForSpam()
NB_1.train(train_hams, train_spams)

# Build Naive Bayes classifier using "train2()" function
NB_2 = NaiveBayesForSpam()
NB_2.train2(train_hams, train_spams)

**7.** After training the two Naive Bayes classifiers, we examine how each classifier performs out of sample by using the validation set.

In [7]:
# Get messages and labels of the validation set 
train_messages = list(train["Clear Message"])
train_labels = list(train["Type"])

# Get messages and labels of the validation set 
validation_messages = list(validation["Clear Message"])
validation_labels = list(validation["Type"])

In [8]:
%%time
# Execution time of score() function for NB_1 classifier on the training set
acc_train_1, cm_train_1 = NB_1.score(train_messages, train_labels)

Wall time: 3min 52s


In [9]:
%%time
# Execution time of score() function for NB_1 classifier on the validation set
acc_validation_1, cm_validation_1 = NB_1.score(validation_messages, validation_labels)

Wall time: 1min 34s


In [10]:
%%time
# Execution time of score() function for NB_2 classifier on the training set
acc_train_2, cm_train_2 = NB_2.score(train_messages, train_labels)

Wall time: 9.47 s


In [11]:
%%time
# Execution time of score() function for NB_2 classifier on the validation set
acc_validation_2, cm_validation_2 = NB_2.score(validation_messages, validation_labels)

Wall time: 3.78 s


The accuracy of the first classifier **NB_1** is:

**`train()`**:

In [12]:
print("Training set: " + str(acc_train_1) + "\n" + \
      "Validation set: " + str(acc_validation_1))

Training set: 0.97
Validation set: 0.962


The accuracy of the second classifier **NB_2** is:

**`train2()`**:

In [13]:
print("Training set: " + str(acc_train_2) + "\n" + \
      "Validation set: " + str(acc_validation_2))

Training set: 0.9728
Validation set: 0.967


For the out-of-sample performance, refer to the above accuracies pertaining to the validation set. As we can observe, **NB_2** classifier has slight improvement of accuracy over **NB_1**.

**8.** Execution Time and Accuracy of `train2()` Naive Bayes Classifier

After using the two classifiers on both the training and the validation set, we will evaluate them both on their execution time and their accuracy.

#### Execution Time

The execution time of the `score()` function by **NB_1** classifier (which is trained using the `train()` function) is roughly **00:03:52** for the training set of 2,500 messages and **00:01:34** for the validation set of 1,000 messages. For **NB_2** classifier (which is trained using `train2()` function), the execution time is almost **00:00:09** for the training set and approximately **00:00:04** for the validation set. We see that **NB_2** classifier (or `train2()` classifier) requires significantly less time to be executed than `train()` classifier does. 

This difference in execution times is expected. For the **NB_2** classifier (or `train2()` classifier), the likelihoods we used to calculate the posterior probabilities (of a new message being ham or spam) are only tho who satisfy the inequality $20 \times P(W \mid ham) < P(W \mid spam)$, which are far less that the total number of words. We can easily see this from the below:

In [14]:
print("Number of predictor words for NB_1 classifier (train() classifier): ", len(NB_1.words))
print("Number of predictor words for NB_2 classifier (train2() classifier): ", len(NB_2.words))

Number of predictor words for NB_1 classifier (train() classifier):  5463
Number of predictor words for NB_2 classifier (train2() classifier):  231


As we can observe, the number of predictor words used in **NB_2** classifier (or `train2()` classifier) is a lot less. As a result, there are a lot less multiplications to perform. Therefore, **NB_2** classifier is a lot faster in the classification phase.

#### Accuracy

From the results in section 7, we can observe that **NB_2** classifier (or `train2()` classifier) has higher accuracy in both training and validation sets. As discussed, the `train2()` classifier only uses the spam keywords (word whose probability of appearing in spam messages is more than 20 times higher than the probability of appearing in ham messages). This has two effects in the predictions of new messages:

1. **Removal of predictor words with very low predictive power**  
The `train2()` classifier discards the predictor words where the two likelihoods $P(W \mid ham)$ and $P(W \mid spam)$ have very similar value. These can include stopwords (such as 'the', 'a', 'for') which are almost equally likely to be in both types of messages. The appearance (or non-appearance) of these words in the new message does not tell us whether the message is likely to be ham or spam. Thus, the removal of those predictor words can potentially reduce our classification error.  

2. **Removal of other spam and ham keywords**
The `train2()` classifier also discards all the ham keywords and some of the spam keywords (e.g. words that are at least 10 times more likely to appear in spam than in ham). This reduction causes the classifier to be less predictive in detecting spam messages, because the non-appearance of ham keywords (and the appearance of other spam keywords) could result in higher posterior probability of a message being spam. As a result, more messages might be classified as ham using `train2()` classifier. This drives up both the number of True Negatives (Ham messages correctly specified) and False Negatives (Spam messages misclassified as Ham). Due to the fact that there are much more Ham messages than Spam messages in both training and validation sets (and also the overall dataset), the increase in True Negatives more than offsets the decrease in True Positives. Consequently, the final accuracy is higher. This phenomenon can be easily observed from the below confusion matrices.


In [15]:
# Specify format to show no decimal points
pd.options.display.float_format = '{:,.0f}'.format

train_cm1 = pd.DataFrame.from_items([("ham", cm_train_1[0]), \
                                     ("spam", cm_train_1[1])], \
                              orient = "index", columns = ["ham", "spam"])
train_cm1.columns.name, train_cm1.index.name = "Actual values", "Predicted values"
train_cm1

Actual values,ham,spam
Predicted values,,
ham,"2,115",30
spam,45,310


The above is the confusion matrix of the `train()` classifier on training set.

In [16]:
train_cm2 = pd.DataFrame.from_items([("ham", cm_train_2[0]), \
                                     ("spam", cm_train_2[1])], \
                              orient = "index", columns = ["ham", "spam"])
train_cm2.columns.name, train_cm2.index.name = "Actual values", "Predicted values"
train_cm2

Actual values,ham,spam
Predicted values,,
ham,"2,156",64
spam,4,276


Above is the confusion matrix of the `train2()` classifier on training set. Compared to the `train()` classifier, we can observe:

* Higher number in the first row (predicted ham messages): This results in higher True Negatives (Ham messages correctly specified) and False Negatives (Spam messages misclassified as Ham).
* Lower number in the second row (predicted spam messages): This results in lower True Positives (Spam messages correctly specified) and False Positives (Ham messages misclassified as Spam).

The increase in True Negatives is larger than the decrease in True Positives, therefore the `train2()` classifier achieves higher accuracy.

We can observe the similar behaviour in validation set:

In [17]:
cm1 = pd.DataFrame.from_items([("ham", cm_validation_1[0]), \
                               ("spam", cm_validation_1[1])], \
                              orient = "index", columns = ["ham", "spam"])
cm1.columns.name, cm1.index.name = "Actual values", "Predicted values"
cm1

Actual values,ham,spam
Predicted values,,
ham,845,15
spam,23,117


Above is the confusion matrix of `train()` classifier on validation set.

In [18]:
cm2 = pd.DataFrame.from_items([("ham", cm_validation_2[0]), \
                               ("spam", cm_validation_2[1])], \
                              orient = "index", columns = ["ham", "spam"])
cm2.columns.name, cm2.index.name = "Actual values", "Predicted values"
cm2

Actual values,ham,spam
Predicted values,,
ham,859,24
spam,9,108


Above is the confusion matrix of `train2()` classifier on validation set. 

We can observe similar behaviour exhibited in the training set. The increase in True Negatives more than offsets the decrease in True Positives, therefore train2() classifier achieves higher accuracy in the validation set.

**9.** Number of False Positives and Way to Reduce It

We will use the confusion matrix to examine classification results. We shall focus on the number of false positives (ham messages classified as spam  messages) and false negatives (spam messages classified as ham messages).

The confusion matrix of the **NB_1** classifier (or `train()` classifier) in the validation set is:

In [19]:
cm1

Actual values,ham,spam
Predicted values,,
ham,845,15
spam,23,117


The number of false positives (ham messages classified as spam messages) in the above matrix is 23 messages.

The confusion matrix of the **NB_2** classifier (or `train2()` classifier) in the validation set is:

In [20]:
cm2

Actual values,ham,spam
Predicted values,,
ham,859,24
spam,9,108


The number of false positives (ham messages classified as spam messages) in the above matrix is 9 messages.

#### Reduce False Positives at the Expense of Possibly Having More False Negatives

To reduce the number of false positives (ham messages classified as spam messages), we can lower the threshold value in third last line of `predict()` function defined in the Naive Bayes class.

In that line, the current code has **if posteriors[0] > 0.5**. We can reduce this threshold by choosing a value lower than 0.5. This will result in more messages being classified as ham messages, since the messages with lower posterior probability (of being ham) will now also be classified as ham. Consequently, this will reduce the number of false positives (ham messages classified as spam messages) but may drive up the number of false negatives (spam messages classified as ham messages).


**10.** Test Set Performance

Between the two classifiers, we see that the `train2()` classifier has higher accuracy. Thus, we will choose this classifier and proceed to estimate its performance using the test set. Before doing so, we will train the `train2()` classifier on both the training and the validation set. By retraining the model using both training and validation set, the future performance can be more accurately estimated (compared to the scenario where only the training set is used), because we will eventually train the model on the full sample of data to maximise its performance.


In [21]:
# Joining messages of training and validation set of hams and spams
train_and_validation_hams = train_hams + list(validation[validation["Type"] == "ham"]["Clear Message"])
train_and_validation_spams = train_spams + list(validation[validation["Type"] == "spam"]["Clear Message"])

# Build Naive Bayes classifier using "train2()" function
NB_3 = NaiveBayesForSpam()
NB_3.train2(train_and_validation_hams, train_and_validation_spams)

# Get messages and labels of the test set 
test_messages = list(test["Clear Message"])
test_labels = list(test["Type"])

The confusion matrix of the test performance results is as below:

In [22]:
# "train2()" classifier on the test set
acc_test_2, cm_test_2 = NB_3.score(test_messages, test_labels)

cm2_t = pd.DataFrame.from_items([("ham", cm_test_2[0]), \
                                 ("spam", cm_test_2[1])], \
                                orient = "index", columns = ["ham", "spam"])
cm2_t.columns.name, cm2_t.index.name = "Actual values", "Predicted values"
cm2_t

Actual values,ham,spam
Predicted values,,
ham,"1,788",40
spam,11,235


The accuracy of the `train2()` classifier on test set is as below:

In [23]:
print("Test set: " + str(acc_test_2))

Test set: 0.975409836066
